### 1. Scrape Player Rankings

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np

from get_data import get_rankings, scrape_qb, scrape_rb, scrape_wr, scrape_te, scrape_dst, scrape_k
from scoring import get_scoring_dict, qb_scoring, rb_scoring, wr_scoring, te_scoring, dst_scoring, k_scoring
from model import add_player, drop_player, find_player, simulate

In [2]:
player_rankings = get_rankings()
player_rankings.head()

,rank,player,position,team,bye,posrank
0,1,Jonathan Taylor,R,IND,14,1
4,2,Derrick Henry,R,TEN,6,2
8,3,Cooper Kupp,W,LAR,7,1
12,4,Christian McCaffrey,R,CAR,13,3
16,5,Justin Jefferson,W,MIN,7,2


In [3]:
player_rankings.shape

(300, 6)

### 2. Scrape Projections

#### QB Projections

In [4]:
qb_projections = scrape_qb()
qb_projections.head()

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Josh Allen,607.4,390.1,4404.9,34.6,13.9,117.3,663.5,6.8,3.8
1,Patrick Mahomes II,634.2,417.8,4842.0,36.0,11.7,66.5,359.6,2.8,2.8
2,Justin Herbert,625.3,415.3,4682.0,34.0,13.3,63.9,294.6,3.2,1.4
3,Lamar Jackson,520.5,336.7,3733.0,24.5,13.2,158.7,956.1,4.9,3.5
4,Kyler Murray,570.5,383.9,4254.6,27.3,12.4,107.3,552.0,5.5,2.3


Standardize names

In [5]:
qb_projections

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Josh Allen,607.4,390.1,4404.9,34.6,13.9,117.3,663.5,6.8,3.8
1,Patrick Mahomes II,634.2,417.8,4842.0,36.0,11.7,66.5,359.6,2.8,2.8
2,Justin Herbert,625.3,415.3,4682.0,34.0,13.3,63.9,294.6,3.2,1.4
3,Lamar Jackson,520.5,336.7,3733.0,24.5,13.2,158.7,956.1,4.9,3.5
4,Kyler Murray,570.5,383.9,4254.6,27.3,12.4,107.3,552.0,5.5,2.3
5,Jalen Hurts,494.8,307.1,3644.7,24.4,12.6,144.6,772.9,7.9,2.9
6,Dak Prescott,593.6,399.3,4545.3,32.7,11.7,55.8,230.6,2.9,4.1
7,Joe Burrow,570.3,386.6,4576.1,31.3,12.8,46.6,187.4,2.2,1.8
8,Aaron Rodgers,558.2,376.6,4322.7,32.7,7.1,38.7,136.2,1.8,1.6
9,Matthew Stafford,586.7,392.1,4586.4,33.5,13.9,33.3,99.2,1.2,2.4


In [6]:
qb_projections['player_stat'].replace('Patrick Mahomes II','Patrick Mahomes',inplace=True)
# deshaun watson missing from projections

In [7]:
player_rankings[(player_rankings['position']=='Q')&(player_rankings['posrank']==28)]

,rank,player,position,team,bye,posrank
107,267,Deshaun Watson,Q,CLE,9,28


In [8]:
player_rankings.merge(qb_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,43,Josh Allen,Q,BUF,7,1,Josh Allen,607.4,390.1,4404.9,34.6,13.9,117.3,663.5,6.8,3.8
1,53,Patrick Mahomes,Q,KC,8,2,Patrick Mahomes,634.2,417.8,4842.0,36.0,11.7,66.5,359.6,2.8,2.8
2,54,Justin Herbert,Q,LAC,8,3,Justin Herbert,625.3,415.3,4682.0,34.0,13.3,63.9,294.6,3.2,1.4
3,62,Lamar Jackson,Q,BAL,10,4,Lamar Jackson,520.5,336.7,3733.0,24.5,13.2,158.7,956.1,4.9,3.5
4,63,Kyler Murray,Q,ARI,13,5,Kyler Murray,570.5,383.9,4254.6,27.3,12.4,107.3,552.0,5.5,2.3
5,76,Jalen Hurts,Q,PHI,7,6,Jalen Hurts,494.8,307.1,3644.7,24.4,12.6,144.6,772.9,7.9,2.9
6,89,Dak Prescott,Q,DAL,9,7,Dak Prescott,593.6,399.3,4545.3,32.7,11.7,55.8,230.6,2.9,4.1
7,91,Aaron Rodgers,Q,GB,14,9,Aaron Rodgers,558.2,376.6,4322.7,32.7,7.1,38.7,136.2,1.8,1.6
8,92,Russell Wilson,Q,DEN,9,10,Russell Wilson,540.3,356.1,4109.3,29.1,9.6,62.7,278.8,2.1,2.3
9,93,Joe Burrow,Q,CIN,10,11,Joe Burrow,570.3,386.6,4576.1,31.3,12.8,46.6,187.4,2.2,1.8


Add scoring

In [9]:
scoring_dict = get_scoring_dict()
qb_projections = qb_scoring(qb_projections,scoring_dict)
qb_projections.head()

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Josh Allen,607.4,390.1,4404.9,34.6,13.9,117.3,663.5,6.8,3.8,386.346
1,Patrick Mahomes,634.2,417.8,4842.0,36.0,11.7,66.5,359.6,2.8,2.8,361.440
2,Justin Herbert,625.3,415.3,4682.0,34.0,13.3,63.9,294.6,3.2,1.4,342.540
3,Lamar Jackson,520.5,336.7,3733.0,24.5,13.2,158.7,956.1,4.9,3.5,338.930
4,Kyler Murray,570.5,383.9,4254.6,27.3,12.4,107.3,552.0,5.5,2.3,338.184


#### RB Projections

In [10]:
rb_projections = scrape_rb()
rb_projections.head()

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
0,Jonathan Taylor,326.7,1603.4,15.0,42.4,348.4,2.0,1.6
1,Derrick Henry,362.5,1621.2,13.0,26.7,213.5,1.1,1.0
2,Christian McCaffrey,240.7,1073.2,8.1,88.3,749.6,4.1,0.5
3,Austin Ekeler,219.8,954.1,9.0,71.5,633.1,4.7,0.7
4,Joe Mixon,288.2,1203.2,9.5,46.1,339.1,2.4,0.9


Standardize names

In [11]:
rb_projections.iloc[50:100,]

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
50,Boston Scott,69.3,296.3,2.7,8.8,64.1,0.2,0.4
51,Ronald Jones II,53.1,261.2,2.7,9.3,62.5,0.2,0.5
52,Isaiah Spiller,61.4,245.7,1.8,12.4,92.2,0.4,0.2
53,Eno Benjamin,60.0,243.3,1.7,12.3,98.1,0.3,0.3
54,Matt Breida,65.8,271.8,1.3,12.1,91.0,0.5,0.7
55,Samaje Perine,57.3,215.6,1.2,19.9,139.4,0.6,0.2
56,Isiah Pacheco,55.5,224.3,2.0,10.6,71.3,0.4,0.3
57,J.J. Taylor,0.0,235.9,2.0,7.8,57.2,0.3,0.2
58,Jeff Wilson Jr.,46.1,172.6,1.6,11.2,83.5,0.6,0.3
59,D'Ernest Johnson,34.2,197.4,1.4,9.7,77.0,0.2,0.1


In [12]:
# deejay dalls
# dwayne washington
# rico dowdie
# kene nwangwu
# snoop conner
# qb_projections['player_stat'].replace('Patrick Mahomes II','Patrick Mahomes',inplace=True)

In [13]:
player_rankings[(player_rankings['position']=='R')&(player_rankings['posrank']==90)]

,rank,player,position,team,bye,posrank
171,283,Larry Rountree III,R,LAC,8,90


In [14]:
player_rankings.merge(rb_projections,left_on='player',right_on='player_stat').iloc[80:90,]

,rank,player,position,team,bye,posrank,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles


In [15]:
rb_projections = rb_scoring(rb_projections,scoring_dict)
rb_projections.head()

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles,points
0,Jonathan Taylor,326.7,1603.4,15.0,42.4,348.4,2.0,1.6,293.98
1,Derrick Henry,362.5,1621.2,13.0,26.7,213.5,1.1,1.0,266.07
2,Christian McCaffrey,240.7,1073.2,8.1,88.3,749.6,4.1,0.5,254.48
3,Austin Ekeler,219.8,954.1,9.0,71.5,633.1,4.7,0.7,239.52
4,Joe Mixon,288.2,1203.2,9.5,46.1,339.1,2.4,0.9,223.83


### WR Projections

In [16]:
wr_projections = scrape_wr()
wr_projections.head()

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
0,Cooper Kupp,120.9,1560.6,12.1,4.1,25.1,0.1,0.4
1,Ja'Marr Chase,92.2,1408.7,10.6,8.1,52.5,0.2,0.4
2,Justin Jefferson,102.8,1471.9,9.6,2.6,17.0,0.1,0.3
3,Deebo Samuel,70.2,1014.7,6.4,67.1,381.5,4.4,0.6
4,Davante Adams,105.5,1302.4,9.8,0.0,0.0,0.0,0.2


Standardize names

In [17]:
wr_projections[60:120]

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
60,Jahan Dotson,52.4,673.2,3.7,0.0,0.0,0.0,0.2
61,George Pickens,53.7,663.4,3.7,0.0,0.0,0.0,0.2
62,Van Jefferson,45.8,618.5,4.0,1.8,12.7,0.1,0.2
63,Curtis Samuel,44.1,513.3,3.5,15.6,89.8,0.8,0.2
64,Jalen Tolbert,46.6,601.6,4.1,0.0,0.0,0.0,0.2
65,Donovan Peoples-Jones,43.3,620.9,3.4,0.0,0.0,0.0,0.2
66,Parris Campbell,49.1,598.1,3.7,0.5,3.3,0.0,0.1
67,Rondale Moore,51.6,513.2,2.6,16.5,103.0,0.8,0.4
68,Joshua Palmer,46.9,550.8,4.3,0.0,0.0,0.0,0.2
69,KJ Hamler,44.1,578.4,3.4,2.6,19.8,0.0,0.2


In [18]:
wr_projections['player_stat'].replace('Joshua Palmer','Josh Palmer',inplace=True)
wr_projections['player_stat'].replace('Cedrick Wilson Jr.','Cedrick Wilson',inplace=True)
# velus jones jr.
# obj
# d'wayne eskridge
# terrace marshall jr.
# bryan edwards

In [19]:
player_rankings[(player_rankings['position']=='W')&(player_rankings['posrank']==97)]

,rank,player,position,team,bye,posrank
123,271,Terrace Marshall Jr.,W,CAR,13,97


In [20]:
player_rankings.merge(wr_projections,left_on='player',right_on='player_stat')[80:100]

,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
80,233,Chris Conley,W,HOU,6,89,Chris Conley,24.5,297.2,2.0,0.0,0.0,0.0,0.0
81,234,Parris Campbell,W,IND,14,90,Parris Campbell,49.1,598.1,3.7,0.5,3.3,0.0,0.1
82,252,Devin Duvernay,W,BAL,10,92,Devin Duvernay,39.4,445.6,2.3,4.9,33.8,0.1,0.2
83,260,Donovan Peoples-Jones,W,CLE,9,93,Donovan Peoples-Jones,43.3,620.9,3.4,0.0,0.0,0.0,0.2
84,261,Bryan Edwards,W,ATL,14,94,Bryan Edwards,35.7,519.7,2.5,0.3,1.8,0.0,0.1
85,263,Braxton Berrios,W,NYJ,10,96,Braxton Berrios,32.7,327.4,1.6,3.0,18.7,0.1,0.2
86,271,Terrace Marshall Jr.,W,CAR,13,97,Terrace Marshall Jr.,23.7,293.6,1.9,0.0,0.0,0.0,0.1
87,272,Zay Jones,W,JAC,11,98,Zay Jones,42.3,510.4,3.1,0.0,0.0,0.0,0.2
88,275,Jauan Jennings,W,SF,9,101,Jauan Jennings,28.0,324.3,2.2,0.0,0.0,0.0,0.1
89,295,Nelson Agholor,W,NE,10,104,Nelson Agholor,28.7,400.2,2.2,1.1,6.6,0.1,0.1


In [21]:
wr_projections = wr_scoring(wr_projections,scoring_dict)
wr_projections.head()

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Cooper Kupp,120.9,1560.6,12.1,4.1,25.1,0.1,0.4,230.97
1,Ja'Marr Chase,92.2,1408.7,10.6,8.1,52.5,0.2,0.4,210.12
2,Justin Jefferson,102.8,1471.9,9.6,2.6,17.0,0.1,0.3,206.49
3,Deebo Samuel,70.2,1014.7,6.4,67.1,381.5,4.4,0.6,203.22
4,Davante Adams,105.5,1302.4,9.8,0.0,0.0,0.0,0.2,188.64


### TE Projections

In [22]:
te_projections = scrape_te()
te_projections.head()

,player_stat,receptions,receive_yards,receive_tds,fumbles
0,Mark Andrews,94.8,1182.3,8.7,0.3
1,Travis Kelce,96.0,1134.7,9.0,0.3
2,Kyle Pitts,75.0,989.6,5.0,0.2
3,Darren Waller,74.9,907.0,5.3,0.2
4,Dalton Schultz,76.9,800.4,5.9,0.2


Standardize names

In [23]:
te_projections

,player_stat,receptions,receive_yards,receive_tds,fumbles
0,Mark Andrews,94.8,1182.3,8.7,0.3
1,Travis Kelce,96.0,1134.7,9.0,0.3
2,Kyle Pitts,75.0,989.6,5.0,0.2
3,Darren Waller,74.9,907.0,5.3,0.2
4,Dalton Schultz,76.9,800.4,5.9,0.2
...,...,...,...,...,...
75,Jack Stoll,7.1,73.7,0.3,0.0
76,Tanner Hudson,5.7,61.9,0.4,0.1
77,Jesper Horsted,5.4,57.1,0.5,0.0
78,Colby Parkinson,5.4,52.2,0.4,0.1


In [24]:
player_rankings.merge(te_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,receptions,receive_yards,receive_tds,fumbles
0,19,Mark Andrews,T,BAL,10,1,Mark Andrews,94.8,1182.3,8.7,0.3
1,21,Travis Kelce,T,KC,8,2,Travis Kelce,96.0,1134.7,9.0,0.3
2,38,Kyle Pitts,T,ATL,14,3,Kyle Pitts,75.0,989.6,5.0,0.2
3,60,Darren Waller,T,LV,6,4,Darren Waller,74.9,907.0,5.3,0.2
4,61,George Kittle,T,SF,9,5,George Kittle,67.3,817.2,5.4,0.3
5,77,Dalton Schultz,T,DAL,9,6,Dalton Schultz,76.9,800.4,5.9,0.2
6,78,T.J. Hockenson,T,DET,6,7,T.J. Hockenson,67.9,697.8,5.2,0.2
7,87,Dallas Goedert,T,PHI,7,8,Dallas Goedert,63.5,772.8,4.8,0.2
8,88,Zach Ertz,T,ARI,13,9,Zach Ertz,63.3,650.8,4.5,0.1
9,109,Hunter Henry,T,NE,10,10,Hunter Henry,55.3,631.2,5.8,0.1


In [25]:
te_projections = te_scoring(te_projections,scoring_dict)
te_projections.head()

,player_stat,receptions,receive_yards,receive_tds,fumbles,points
0,Mark Andrews,94.8,1182.3,8.7,0.3,169.83
1,Travis Kelce,96.0,1134.7,9.0,0.3,166.87
2,Kyle Pitts,75.0,989.6,5.0,0.2,128.56
3,Darren Waller,74.9,907.0,5.3,0.2,122.10
4,Dalton Schultz,76.9,800.4,5.9,0.2,115.04


### D/ST Projections

In [26]:
dst_projections = scrape_dst()
dst_projections.head()

C:\Users\Dan\Desktop\Documents\Misc Documents\Sports\Football\TLOEB\get_data.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dst_projections['player_stat'][i] = dst_projections['player_stat'][i].split(' ')[-1] + ' D/ST'


,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
0,Buccaneers D/ST,45.0,14.4,11.8,2.9,0.6,366.7,"5,536.3"
1,Steelers D/ST,49.3,14.4,10.3,2.5,0.2,392.3,"5,848.1"
2,Bills D/ST,43.1,16.3,10.0,2.8,0.6,331.6,"5,289.3"
3,Packers D/ST,43.3,16.5,9.6,2.8,0.2,354.0,"5,593.0"
4,Cowboys D/ST,40.1,16.5,10.4,3.0,0.3,373.2,"5,838.5"


In [27]:
player_rankings.merge(dst_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
0,141,Bills D/ST,D,BUF,7,1,Bills D/ST,43.1,16.3,10.0,2.8,0.6,331.6,"5,289.3"
1,142,Saints D/ST,D,NO,14,2,Saints D/ST,44.4,15.1,9.8,2.6,0.2,341.4,"5,598.8"
2,143,Packers D/ST,D,GB,14,3,Packers D/ST,43.3,16.5,9.6,2.8,0.2,354.0,"5,593.0"
3,144,Ravens D/ST,D,BAL,10,4,Ravens D/ST,38.5,13.6,9.5,2.6,0.6,358.3,"5,766.1"
4,145,Buccaneers D/ST,D,TB,11,5,Buccaneers D/ST,45.0,14.4,11.8,2.9,0.6,366.7,"5,536.3"
5,146,49ers D/ST,D,SF,9,6,49ers D/ST,44.7,13.7,10.5,2.5,0.2,359.8,"5,516.6"
6,147,Cowboys D/ST,D,DAL,9,7,Cowboys D/ST,40.1,16.5,10.4,3.0,0.3,373.2,"5,838.5"
7,148,Colts D/ST,D,IND,14,8,Colts D/ST,36.2,15.2,11.3,2.7,0.9,364.8,"5,716.3"
8,149,Steelers D/ST,D,PIT,9,9,Steelers D/ST,49.3,14.4,10.3,2.5,0.2,392.3,"5,848.1"
9,150,Commanders D/ST,D,WAS,14,10,Commanders D/ST,39.8,13.2,10.0,2.5,0.2,389.3,"5,861.5"


In [28]:
dst_projections = dst_scoring(dst_projections,scoring_dict)
dst_projections.head()

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against,points
0,Buccaneers D/ST,45.0,14.4,11.8,2.9,0.6,366.7,"5,536.3",18.228571
1,Steelers D/ST,49.3,14.4,10.3,2.5,0.2,392.3,"5,848.1",9.014286
2,Bills D/ST,43.1,16.3,10.0,2.8,0.6,331.6,"5,289.3",25.957143
3,Packers D/ST,43.3,16.5,9.6,2.8,0.2,354.0,"5,593.0",18.557143
4,Cowboys D/ST,40.1,16.5,10.4,3.0,0.3,373.2,"5,838.5",12.871429


### K Projections

In [29]:
k_projections = scrape_k()
k_projections.head()

,player_stat,fgm,fga,xp
0,Justin Tucker,30.0,34.2,39.9
1,Harrison Butker,27.8,31.7,46.3
2,Daniel Carlson,30.6,35.8,37.3
3,Matt Gay,29.2,34.4,41.3
4,Tyler Bass,27.0,33.3,45.6


In [30]:
k_projections[0:10]

,player_stat,fgm,fga,xp
0,Justin Tucker,30.0,34.2,39.9
1,Harrison Butker,27.8,31.7,46.3
2,Daniel Carlson,30.6,35.8,37.3
3,Matt Gay,29.2,34.4,41.3
4,Tyler Bass,27.0,33.3,45.6
5,Greg Joseph,29.8,34.9,36.9
6,Matt Prater,27.0,33.7,43.6
7,Nick Folk,28.6,33.3,38.7
8,Evan McPherson,28.8,34.6,38.2
9,Brandon McManus,27.1,32.8,40.2


In [31]:
k_projections

,player_stat,fgm,fga,xp
0,Justin Tucker,30.0,34.2,39.9
1,Harrison Butker,27.8,31.7,46.3
2,Daniel Carlson,30.6,35.8,37.3
3,Matt Gay,29.2,34.4,41.3
4,Tyler Bass,27.0,33.3,45.6
5,Greg Joseph,29.8,34.9,36.9
6,Matt Prater,27.0,33.7,43.6
7,Nick Folk,28.6,33.3,38.7
8,Evan McPherson,28.8,34.6,38.2
9,Brandon McManus,27.1,32.8,40.2


In [32]:
player_rankings.merge(k_projections,left_on='player',right_on='player_stat')

,rank,player,position,team,bye,posrank,player_stat,fgm,fga,xp
0,151,Justin Tucker,K,BAL,10,1,Justin Tucker,30.0,34.2,39.9
1,152,Evan McPherson,K,CIN,10,2,Evan McPherson,28.8,34.6,38.2
2,153,Matt Gay,K,LAR,7,3,Matt Gay,29.2,34.4,41.3
3,154,Harrison Butker,K,KC,8,4,Harrison Butker,27.8,31.7,46.3
4,155,Daniel Carlson,K,LV,6,5,Daniel Carlson,30.6,35.8,37.3
5,156,Brandon McManus,K,DEN,9,6,Brandon McManus,27.1,32.8,40.2
6,157,Nick Folk,K,NE,10,7,Nick Folk,28.6,33.3,38.7
7,158,Tyler Bass,K,BUF,7,8,Tyler Bass,27.0,33.3,45.6
8,159,Matt Prater,K,ARI,13,9,Matt Prater,27.0,33.7,43.6
9,160,Graham Gano,K,NYG,9,10,Graham Gano,27.4,32.6,30.5


In [33]:
k_projections = k_scoring(k_projections,scoring_dict)
k_projections.head()

,player_stat,fgm,fga,xp,points
0,Justin Tucker,30.0,34.2,39.9,153.705
1,Harrison Butker,27.8,31.7,46.3,151.285
2,Daniel Carlson,30.6,35.8,37.3,152.635
3,Matt Gay,29.2,34.4,41.3,150.835
4,Tyler Bass,27.0,33.3,45.6,145.020


### Combine projections

In [34]:
all_projections = pd.concat([qb_projections[['player_stat','points']],rb_projections[['player_stat','points']],wr_projections[['player_stat','points']],
          te_projections[['player_stat','points']],dst_projections[['player_stat','points']],k_projections[['player_stat','points']]])

full_df = player_rankings.merge(all_projections,left_on='player',right_on='player_stat')
full_df.sort_values(by='points',ascending=False,inplace=True)
full_df.head()

,rank,player,position,team,bye,posrank,player_stat,points
40,43,Josh Allen,Q,BUF,7,1,Josh Allen,386.346
49,53,Patrick Mahomes,Q,KC,8,2,Patrick Mahomes,361.440
50,54,Justin Herbert,Q,LAC,8,3,Justin Herbert,342.540
58,62,Lamar Jackson,Q,BAL,10,4,Lamar Jackson,338.930
59,63,Kyler Murray,Q,ARI,13,5,Kyler Murray,338.184


### Develop Model

Initialize draft df and my team

In [35]:
draft_df = full_df.copy()

my_team = full_df.copy()
my_team = my_team[my_team['player']=='a']
my_team

# full_df.to_csv('draft_ppr.csv',index=False)

,rank,player,position,team,bye,posrank,player_stat,points


## Draft admin

Drop, simulate, find, draft, add, drop

In [36]:
print(len(draft_df))
draft_df = drop_player(draft_df,"")
print(len(draft_df))

256
256


In [37]:
output = simulate(my_team,draft_df,simulations=150)
output.columns = ['count','min','mean','max']
output.sort_values(by='mean',ascending=False)

Time:  43.6806128


,count,min,mean,max
Rnd1,,,,
R,16,1459.0,1528.0,1604.0
W,28,1411.0,1506.0,1633.0
T,25,1410.0,1483.0,1563.0
Q,24,1428.0,1478.0,1552.0
DST,31,1375.0,1453.0,1510.0
K,26,1356.0,1449.0,1518.0


In [38]:
find_player(my_team,draft_df,'R')

,rank,player,position,team,bye,posrank,player_stat,points
0,1,Jonathan Taylor,R,IND,14,1,Jonathan Taylor,293.98
1,2,Derrick Henry,R,TEN,6,2,Derrick Henry,266.07
3,4,Christian McCaffrey,R,CAR,13,3,Christian McCaffrey,254.48
6,7,Austin Ekeler,R,LAC,8,4,Austin Ekeler,239.52
9,10,Joe Mixon,R,CIN,10,7,Joe Mixon,223.83


In [39]:
player = 'Jonathan Taylor'
my_team = add_player(my_team,draft_df,player)
my_team.index = list(range(len(my_team)))

In [40]:
my_team

,rank,player,position,team,bye,posrank,player_stat,points
0,1,Jonathan Taylor,R,IND,14,1,Jonathan Taylor,293.98


In [41]:
print(len(draft_df))
draft_df = drop_player(draft_df,player)
print(len(draft_df))

256
255
